# Find Mask shapes from Cellpose segmentation
------
Author: Ryan Corbyn  
Date: 21/02/2023

--------

### Description: 

This is a script that can be used to find some shape descriptors for segmentation masks generated by the cellpose segmentation program. 

This script takes as inputs from the user: 
1. The stack image file generated by the program: 1_Cell_segmentation_and_tracking
2. The cellpose segmentation masks as a .tif image stack file. (All the masks for each image are contained in a single .tif file)

The btrack algorithm is used to generate the tracks for the cell masks across all time points. The results of this can the be filtered so that only cells which have been tracked for a minimum of, for example 10, frames are seen as valid. The filtered results are saved into a variable called "filtered_tracks". 

The "filtered_tracks" variable is then used to identify each of the cell masks in the time sequence and the shape parameters for the masks calculated. The results of this analysis are then saved into a .csv file for the user. 

------------

### The user is required to input: 
1. The image resolution for the original image. 
2. The minimum track length. 
3. The file location of the time-lapse stack image. 
4. The file location of the cell-mask stack image. 

## Cell 1
-----
### Import Dependancies

-----

In [1]:
#Import cellpose and other dependancies
import btrack

# Import libraries for data analysis
import numpy as np 
import tifffile as tf
import pandas as pd
import os
from skimage import measure

# Import dependancies for user interface/input
from tqdm import tqdm
import tkinter as tk
from tkinter import filedialog

## Cell 2
---
### User inputs
---

In the following cell, the user needs to input: 
1) The resolution of the images to be analysed. 
2) The minimum track length for an object to be included in the analysis.

In [5]:
# Input the image resolution:
image_resolution = 0.635 # microns per pixel. 

# select the minimum number of tracks that an object must be tracked for. 
min_tracks = 3

## Cell 3
----
### Select input folder containing the image dataset
----

In [6]:
# # # Get the folder containing the image files. 
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw() # Stops a second window opening

# Select the file name. 
image_stack = filedialog.askopenfilename(title = 'Select Folder Containing images')

## Cell 4
---
### Choose the file containing the segmentation masks as a stack image.
---

In [21]:
# # # Creates dialogue to ask directory
# # # Get the folder containing the image stack. 
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw() # Stops a second window opening
mask_file_name = filedialog.askopenfilename(title = 'Select Stack file')

print(np_masks.shape)

(1, 1, 97, 1040, 1392)


## Cell 5 
---
### Calculates the cell tracks from the segmentation mask stack file.
---

In [8]:
def find_Btracks(image_masks, configer_file, script_fold):
    '''A method to find the tracks from the cellpose masks
    generated from the image stack. 
    this method makes use of the trackpy package.'''
    
    FEATURES = [
      "area",
      "major_axis_length",
      "minor_axis_length",
       "eccentricity",
      "solidity",
                ]
    
    TRACKING_UPDATES = [
      "motion",
      "visual",
            ]

    objects = btrack.utils.segmentation_to_objects(image_masks, properties=tuple(FEATURES))
    
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure_from_file(configer_file)

        tracker.features = FEATURES

        # append the objects to be tracked
        tracker.append(objects) 

        # set the volume (Z axis volume is set very large for 2D data)
        tracker.volume=((0, 2000), (0, 2000), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track(step_size=100)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        # store the data in an HDF5 file
        tracker.export(script_fold + '/tracks.h5', obj_type='obj_type_1')

        # get the tracks as a python list
        tracks = tracker.track(tracking_updates=TRACKING_UPDATES)

        # get the data in a format for napari
        data, properties, graph = tracker.to_napari(ndim=2)
    
    return(data, properties, graph)

## Cell 6
---
### Filter the tracks by user defined track-length
---

In [9]:
def filter_tracks(tracks, props, min_track_len, indicies):
    '''Filter the tracks variable to only contain tracks longer than a 
    specified minimum (track_min). The resulting pandas array is then 
    sorted by object order.'''
    
    
    filtered_tracks = pd.DataFrame()
    filtered_props = pd.DataFrame()
    
    for i in range(indicies.shape[0]):
        if indicies[i][1] - indicies[i][0] > min_track_len: 
            single_track = pd.DataFrame(tracks.iloc[indicies[i][0]: indicies[i][1]])
            single_props = pd.DataFrame( props.iloc[indicies[i][0]: indicies[i][1]] )
            
            filtered_tracks = pd.concat([filtered_tracks, single_track], ignore_index = True)
            filtered_props = pd.concat([filtered_props, single_props], ignore_index = True)
        
    return(filtered_tracks, filtered_props)

## Cell 7
---
### Extract which of the tracks remain after filtering
---

In [10]:
def particle_index(tracks):
    '''Find the indicies where the particle object in the 
    sorted_tracks variable changes. '''
    
    # Set the initial particle value
    particle = tracks['Track'].iloc[0]
    # generate the index_score variable. 
    index_score = [0]

    # Loop to find the indicies in which the particle value changes. 
    for i in range(tracks.shape[0]):
        if tracks['Track'].iloc[i] != particle:
            index_score.append(i-1)
            particle = tracks['Track'].iloc[i]
            index_score.append(i)

    # grab last i value. 
    index_score.append(i)
    
    # Convert index score to a 2D numpy array. 
    index_score = np.reshape(index_score, [int(len(index_score)/2), 2])
    
    return(index_score)

## Cell 8
---
### Find the perimeter of the segmentation masks. 
---

In [11]:
def get_contours(image):
    '''Use skimage to find the coutours (perimeter) of the cellpose masks.'''
    
    image[image < image_value] = 0
    image[image > image_value] = 0

    contours = measure.find_contours(image, 0.1)
    contours = np.array(contours[0]).astype(int)
        
    return(contours)

## Cell 9
---
### Make sure the sub-image of a single mask is at least 2 pixels away from the boarder of the image
---

In [12]:
def correct_index(index_1, image_cropped): 
    '''Due to some weird calculation problems. I need to either subtrack or add 2 to the 
    dimensions of the sub-image in order to get a good measure of the perimeter of the
    cell mask. 
    
    This method makes sure that the indecies used to create the sub image has at least -/+ 2 
    pixels between the cell mask and the image boudary. '''
    
    if index_1[0] == 1: 
        index_1[0] = 2
    if index_1[1] == image_cropped.shape[0]-1:
        index_1[1] = image_cropped.shape[0]-2
    if index_1[2] == 1: 
        index_1[2] = 2
    if index_1[3] == image_cropped.shape[1]-1:
        index_1[3] = image_cropped.shape[1]-2
        
    return(index_1)

## Cell 10 
---
### Calculate the major and minor axis of the segmentation mask. 
---

In [13]:
def find_shape_values(contours): 
    '''Find the values for the major and minor axis of the cells 
    and the eccentricity of the mask.'''

    half_contour_len = int( ( len(contours)-1 ) / 2 )
    quart_leng = int (half_contour_len / 2)
    max_dist = 0
    min_dist = 0

    for i in range(half_contour_len): 
        x_dist = np.power(contours[i, 1] -  contours[i + half_contour_len, 1], 2)
        y_dist = np.power(contours[i, 0] -  contours[i + half_contour_len, 0], 2)

        total_dist = np.sqrt( x_dist + y_dist ) 

        if total_dist > max_dist: 
            max_dist = total_dist 
            
            a = i + quart_leng
            b = (i + half_contour_len + quart_leng) % len(contour)
            
            x_min_dist = np.power(contours[a, 1] -  contours[b, 1], 2)
            y_min_dist = np.power(contours[a, 0] -  contours[b, 0], 2)

            min_dist = np.sqrt(x_min_dist + y_min_dist)
    
    dist = [min_dist, max_dist]
    
    return(np.max(dist), np.min(dist))

## Cell 11
----
### Calculate the cell tracks from segmentation masks. 
----

In [14]:
stack_masks = np.array(raw_masks)
print(stack_masks.shape)
stack_masks = stack_masks[0, 0, :, :, :]

script_folder = os.getcwd()
configer_file = script_folder + '\\cell_config.json'

# Find the tracks for the individual cells in the image stack. 
cell_tracks, properties, graph = find_Btracks(stack_masks, configer_file, script_folder)

[INFO][2024/02/15 11:30:22 AM] Localizing objects from segmentation...


(1, 1, 97, 1040, 1392)


[INFO][2024/02/15 11:30:41 AM] Objects are of type: <class 'dict'>
[INFO][2024/02/15 11:30:41 AM] ...Found 4576 objects in 97 frames.
[INFO][2024/02/15 11:30:41 AM] Loaded btrack: C:\Users\RCORBYN\Anaconda3\envs\cellpose_and_tracking_gpu\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2024/02/15 11:30:41 AM] btrack (v0.5.0) library imported
[INFO][2024/02/15 11:30:41 AM] Starting BayesianTracker session
[INFO][2024/02/15 11:30:41 AM] Loading configuration file: C:\Users\RCORBYN\OneDrive - University of Glasgow\Desktop\For paper\20231114_Simona\cell_config.json
[INFO][2024/02/15 11:30:41 AM] Objects are of type: <class 'list'>
[INFO][2024/02/15 11:30:41 AM] Starting tracking... 
[INFO][2024/02/15 11:30:41 AM] Update using: ['MOTION']
[INFO][2024/02/15 11:30:41 AM] Tracking objects in frames 0 to 97 (of 97)...
[INFO][2024/02/15 11:30:41 AM]  - Timing (Bayesian updates: 0.00ms, Linking: 0.00ms)
[INFO][2024/02/15 11:30:41 AM]  - Probabilities (Link: 0.99986, Lost: 1.00000)
[INFO][2024/

## Cell 12
----
### Filter the tracks according to user-defined minimum track length
---

In [15]:
# Get the tracks data into a pandas dataframe. 
data_arr = np.array(cell_tracks)
df_data = pd.DataFrame({'Track': data_arr[:, 0].astype(int), 'Frame': data_arr[:, 1].astype(int), 'X_pos': data_arr[:, 2], 'Y_pos': data_arr[:, 3]})

# Find the indicies which describe a change in the object track. 
index = particle_index(df_data)

# Filter the tracks by the minimum track length. 
filtered_tracks, filtered_props = filter_tracks(df_data, pd.DataFrame(properties), min_tracks, index)

## Cell 13
----
### Calculate the shape features for all masks in the filtered tracks list. 
---

In [19]:
# Initialise variable. 
track_shapes_pd = pd.DataFrame()

pixel_area = np.power(image_resolution, 2)

# for all the frames contained in the filtered_tracks variable. 
for i in tqdm( range( len( filtered_tracks['Track'] ) ) ):
    
    # Extract the frame of interest. 
    image_to_analyse = np.array(stack_masks[filtered_tracks['Frame'][i], :, :])
    
    # Extract the [x,y] for the track central point (track point) . 
    y_loc = int(filtered_tracks['X_pos'][i])
    x_loc = int(filtered_tracks['Y_pos'][i] )
    
    # Makle sure the track point is within the image boundry. 
    if x_loc > stack_masks.shape[2]: 
        x_loc = stack_masks.shape[2]-1
    if y_loc > stack_masks.shape[1]:
        y_loc = stack_masks.shape[1]-1
    
    # Find the pixel value at the track point.
    image_value = image_to_analyse[ y_loc, x_loc ]
    
    # Ignore the analysis if the track point value is 0.
    # This means the tracking has missed the cell mask. 
    if image_value == 0: 
        continue 
    else: 
        # Find all the values in the image with the same value as 
        # the track point. 
        find_mask = np.where(image_to_analyse == image_value)
        
        # Find the values needed to generate a square around the 
        # mask in question. 
        index = [np.min(find_mask[0]), np.max(find_mask[0]), 
                            np.min(find_mask[1]), np.max(find_mask[1])]

        # If the cell mask is at the image boarder, ignore it. 
        if index[0] == 0 or index[2] == 0 or np.max(find_mask[0]) == image_to_analyse.shape[0] or np.max(find_mask[1]) == image_to_analyse.shape[1]:
            continue 

        else: 
            # Correct the index value to allow 2 pixels between the 
            # cell mask and the image boarder. 
            # Needed to accurately find the perimeter of the cell mask. 
            index = correct_index(index, image_to_analyse)
            
            # Generate a sub image: a square around the mask in question. 
            sub_image = np.array(image_to_analyse[index[0]-2 : index[1]+2, index[2]-2 : index[3]+2])
            
            # Find the perimeter around the cell. 
            contour = get_contours(sub_image)
            
            # Find the area and perimeter of the cell mask in pixels. 
            area = len(find_mask[0])
            perimeter = len(contour[:,0])
            
            # Find the major and minor axis of the cell mask. 
            # Needed to calculate eccentricity. 
            major_axis, minor_axis = find_shape_values(contour)
            # Calcuate eccentricity. 
            eccent = np.sqrt(1 - (minor_axis / major_axis) )
            
            # Store all information into a dictionary. 
            cell_shape_dict = {'Track': [filtered_tracks['Track'][i]],
                   'Frame': [filtered_tracks['Frame'][i]],
                    'Mask Number': [image_value],
                   'X_pos': [filtered_tracks['Y_pos'][i]], 
                   'Y_pos': [filtered_tracks['X_pos'][i]], 
                   'Area (microns Squared)': [area * pixel_area], 
                   'Perimeter (microns)': [perimeter * image_resolution], 
                   'Major Axis (microns)': [major_axis * image_resolution], 
                   'Minor Axis (microns)': [minor_axis * image_resolution], 
                   'eccentricity': [eccent]}
            
            # Convert dictionary to dataframe
            temp_df = pd.DataFrame(cell_shape_dict)
            
            # Append all information into a local variable. 
            track_shapes_pd = pd.concat([track_shapes_pd, temp_df])



100%|█████████████████████████████████████████████████████████████████████████████| 4466/4466 [00:41<00:00, 108.53it/s]


## Cell 14
----
## Save the segmentation mask data. 
----

In [ ]:
# Extract the folder path from the file containing 
# the segmentation mask stack.
save_path = os.path.dirname(mask_file_name)

# Save the analysis to the same folder as the segmentation mask file. 
track_shapes_pd.to_csv(save_path + '\\' + 'cellpose_mask_shape_analysis.csv',
                       index=False)